In [2]:
!pip install tensorflow gradio

In [5]:
import tensorflow as tf
import numpy as np

# Loading the model
model = tf.keras.models.load_model('/content/drive/My Drive/SM_Digit_Recognizer.h5')

In [40]:
import numpy as np
from PIL import Image, ImageOps

def preprocess_image(image):
    # Invert image if needed
    if np.mean(image) > 0.5:  # Simple heuristic: if average pixel value is high, it's likely a white background
        image = 1.0 - image  # Invert the image
    return image

def predict_digit(image):
    # Convert image to grayscale and handle transparency
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image).convert('L')
    else:
        image = Image.open(image).convert('L')

    # Resize the image to match the input size expected by the model (28x28 pixels)
    image = image.resize((28, 28), Image.LANCZOS)

    # Ensure the background is white if transparent
    image = ImageOps.invert(image) if image.getextrema()[-1] == 0 else image

    # Convert the image to a NumPy array and normalize pixel values to [0, 1]
    image = np.array(image) / 255.0

    # Preprocess the image (e.g., invert colors if needed)
    image = preprocess_image(image)

    # Reshape the image to add batch and channel dimensions: (1, 28, 28, 1)
    image = image.reshape(1, 28, 28, 1)

    # Predict the digit using the model
    prediction = model.predict(image)

    # Find the index of the highest probability (the predicted digit)
    predicted_digit = int(np.argmax(prediction))

    return predicted_digit


In [42]:
import gradio as gr

# Updated Gradio interface
interface = gr.Interface(
    fn=predict_digit,  # The function that takes input and returns output
    inputs=gr.Image(),  # Grayscale image
    # inputs=gr.Sketchpad(),  # Allows users to draw on the canvas
    outputs=gr.Label(),  # Display prediction label
    live=True  # Allow live updating as the user draws
)


# Launch the interface
interface.launch(debug=False)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e6332131ccfc53074d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
